In [11]:
pwd

'/home/wsuser/work'

In [12]:
!pip install keras==2.2.4
!pip install tensorflow==1.14.0

     |████████████████████████████████| 312 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 8.7 MB/s  eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.2 requires keras<2.8,>=2.7.0, but you have keras 2.2.4 which is incompatible.
ERROR: Could not find a version that satisfies the requirement tensorflow==1.14.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2)
ERROR: No 

In [13]:
import os
os.getcwd()

'/home/wsuser/work'

In [14]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Convolution2D , MaxPooling2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

Using TensorFlow backend.


In [15]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_590efa0248834c71b5605d140e540ae3 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='uiDUfTp-XlwYIS81fzGguQDDCYsnwp48kJFXfCrRYb5J',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_5 = client_590efa0248834c71b5605d140e540ae3.get_object(Bucket='imageclassification-donotdelete-pr-ri8tzpkj34bnkn', Key='Coal Classification-20220918T023819Z-001.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [16]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_5.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [17]:
pwd

'/home/wsuser/work'

In [18]:
import cv2,os
data_path='/home/wsuser/work/Coal Classification/train'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories, labels))
print(label_dict)
print(categories)
print(labels)


{'Peat': 0, 'Anthracite': 1, 'Lignite': 2, 'Bituminous': 3}
['Peat', 'Anthracite', 'Lignite', 'Bituminous']
[0, 1, 2, 3]


In [19]:
train_datagen = ImageDataGenerator(rescale=1./ 255,shear_range=0.2,zoom_range=0.2, horizontal_flip=True,validation_split=0.2)

In [20]:
test_datagen = ImageDataGenerator(rescale=1./ 255, )

In [21]:
X_train = train_datagen.flow_from_directory(r"/home/wsuser/work/Coal Classification/train",target_size=(128, 128),batch_size=4, class_mode='categorical')
  

Found 243 images belonging to 4 classes.


In [22]:
X_test = test_datagen.flow_from_directory(r"/home/wsuser/work/Coal Classification/test",target_size=(128, 128),batch_size=4, class_mode='categorical')
  

Found 108 images belonging to 4 classes.


In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

In [24]:
model=Sequential()

In [25]:
model.add(Convolution2D(32,(3,3),input_shape=(128, 128, 3),activation="relu"))


In [26]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [27]:
model.add(Convolution2D(32,(3,3),activation="relu"))

In [28]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [29]:
model.add(Flatten())

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 28800)             0         
Total params: 10,144
Trainable params: 10,144
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.add(Dense(300,activation="relu"))


In [32]:
model.add(Dense(150,activation="relu"))


In [33]:
model.add(Dense(4,activation="softmax"))


In [34]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

In [35]:
history=model.fit_generator(X_train,validation_data=X_test,epochs=30)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1963: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
61/61 [==============================] - 18s 280ms/step - loss: 1.4617 - accuracy: 0.2716 - val_loss: 1.6495 - val_accuracy: 0.2500
Epoch 2/30
61/61 [==============================] - 16s 270ms/step - loss: 1.4047 - accuracy: 0.2922 - val_loss: 1.3798 - val_accuracy: 0.4074
Epoch 3/30
61/61 [==============================] - 16s 266ms/step - loss: 1.3298 - accuracy: 0.3374 - val_loss: 1.3791 - val_accuracy: 0.2963
Epoch 4/30
61/61 [==============================] - 16s 261ms/step - loss: 1.3510 - accuracy: 0.3292 - val_loss: 1.3788 - val_accuracy: 0.2500
Epoch 5/30
61/61 [==============================] - 16s 270ms/step - loss: 1.3189 - accuracy: 0.3292 - val_loss: 1.3878 - val_accuracy: 0.2963
Epoch 6/30
61/61 [==============================] - 16s 256ms/step - loss: 1.3124 - accuracy: 0.3621 - val_loss: 1.4272 - val_accuracy: 0.2963
Epoch 7/30
61/61 [==============================] - 16s 261ms/step - loss: 1.2529 - accuracy: 0.4527 - val_loss: 1.6286 - val_accuracy: 0.3796

In [36]:
model.save("coal1.h5")

In [37]:
!tar -zcvf image-classification-model_new.tgz coal1.h5

coal1.h5


In [38]:
ls -1

coal1.h5
'Coal Classification'/
image-classification-model_new.tgz


In [39]:
!pip install watson-machine-learning-client --upgrade

In [40]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                     "url": "https://us-south.ml.cloud.ibm.com",
                     "apikey":"Zyz5td4wu8nFQiYLFoz5DOZliuBUeLCEX1AQ0TCaWXEr"
                  }
client = APIClient(wml_credentials)

In [41]:
client = APIClient(wml_credentials)

In [44]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [47]:
space_uid = guid_from_space_name(client, 'imageclassification')
print("Space UID= " + space_uid)

Space UID= eb8d490f-e5a1-4291-b7cb-17ffbb56a1b8


In [46]:
client.set.default_space(space_uid)

'SUCCESS'

In [ ]:
client.software_specifications.list()

In [48]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [49]:



model_details = client.repository.store_model(model='image-classification-model_new.tgz', meta_props={
client.repository.ModelMetaNames.NAME:"CNN",
client.repository.ModelMetaNames.TYPE:"tensorflow_rt22.1",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )

model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [54]:
model_id

'50967871-0c05-4e5b-b634-9380fd0e3ab2'

In [58]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'